In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [3]:
TRAIN_DIR = "resource/CNN/notebooks/dataset/cats vs dogs/train"
VALIDATION_DIR = "resource/CNN/notebooks/dataset/cats vs dogs/validation"
TEST_DIR = "resource/CNN/notebooks/dataset/cats vs dogs/test"
batch_size = 64

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)

Found 24500 images belonging to 2 classes.


Found 500 images belonging to 2 classes.


In [12]:
def costum_network():
    model = Sequential()

    # Conv layer 1
    model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(224, 224, 3)))
    model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Conv layer 2
    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Conv layer 3
    model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
    model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
    model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # fc
    model.add(Flatten())
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="softmax"))
    
    return model

def vgg_16():
    vgg = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
    for layer in vgg.layers:
        layer.trainable=False
    x = vgg.layers[-2].output
    output_layer = Dense(2, activation='softmax', name='predictions')(x)

    model = Model(inputs=vgg.input, outputs=output_layer)
    
    return model

def mobile_net():
    mobile_net = MobileNet(include_top=True, pooling='max', input_shape=(224, 224, 3), classes=2, weights=None)
    for layer in mobile_net.layers:
        layer.trainable=False
    x = mobile_net.layers[-2].output
    output_layer = Dense(2, activation='softmax', name='predictions')(x)

    model = Model(inputs=mobile_net.input, outputs=output_layer)
    
    return model

def inception_v3():
    inception_v3 = InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
    for layer in inception_v3.layers:
        layer.trainable=False
    x = inception_v3.layers[-2].output
    output_layer = Dense(2, activation='softmax', name='predictions')(x)

    model = Model(inputs=inception_v3.input, outputs=output_layer)
    
    return model

In [13]:
model = inception_v3()

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_109 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 56, 56, 64)        0         
__________

In [1]:
# model.fit(x_train, y_train, batch_size=64, epochs=30, shuffle=True, validation_split=0.1)
# print(train_generator.next())
# sess = tf.Session()
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

model.save("cats_vs_dog_cnn.h5")

NameError: name 'model' is not defined